In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, GlobalAveragePooling2D
import os
import cv2
import pickle
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Model


from imblearn.under_sampling import RandomUnderSampler
from keras.utils.np_utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time

%load_ext tensorboard

In [2]:
print(f"Tensorflow version is: {tf.__version__}")

Tensorflow version is: 2.4.1


In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
dataPath = "../../data/OCT/OCT2017/"

In [5]:
labels_available = os.listdir(os.path.join(dataPath, "train"))
print("Total Number of Classes Detected :",len(labels_available))

labels_list = ['NORMAL',"CNV","DME","DRUSEN"]

Total Number of Classes Detected : 4


In [6]:
images=[]
y_trn=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "train", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        images.append(cv2.imread(os.path.join(xPath, y)))
        y_trn.append(labels_list.index(x))
    print(x ,end=" ")

y_train = to_categorical(y_trn,len(labels_list))

NORMAL CNV DME DRUSEN 

In [7]:
x_val=[]
y_val=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "val", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        x_val.append(cv2.imread(os.path.join(xPath, y)))
        y_val.append(labels_list.index(x))
    print(x ,end=" ")

y_validation = to_categorical(y_val,len(labels_list))

NORMAL CNV DME DRUSEN 

In [8]:
x_tst=[]
y_tst=[]
for x in labels_list:
    xPath = os.path.join(dataPath, "test", x)
    myPicList = os.listdir(xPath)
    for y in myPicList:
        x_tst.append(cv2.imread(os.path.join(xPath, y)))
        y_tst.append(labels_list.index(x))
    print(x ,end=" ")

NORMAL CNV DME DRUSEN 

Number of images per category and set

In [9]:
def getNumberOfItems(labels, itemLabel):
    return sum(list(map(lambda x: x == itemLabel, labels))) 


print("Label\t\tTrain\t\tValidation\t\tTest")
for i, label in enumerate(labels_list):
    print(f"{label}\t\t{getNumberOfItems(y_trn, i)}\t\t{getNumberOfItems(y_val, i)}\t\t\t{getNumberOfItems(y_tst, i)}")

Label		Train		Validation		Test
NORMAL		26315		8			242
CNV		37205		8			242
DME		11348		8			242
DRUSEN		8616		8			242


In [10]:
def resizeIm(im, size):
    if im.shape[2] == 1:
        im = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
    return cv2.resize(im, size)

def resizeIms(x, size):
    return np.array(list(map(lambda im: resizeIm(im, size), x)))

# Substract the mean specified by Mean (https://www.mathworks.com/help/deeplearning/ref/nnet.cnn.layer.imageinputlayer.html)
# Substract mean by channel
def zerocenter(x):
    return (x - np.mean(x, axis=(0, 1))) / (2*np.std(x, axis=(0, 1)))

# Rescale the input to be in the range [-1, 1] using the minimum and maximum values specified by Min and Max, respectively. (https://www.mathworks.com/help/deeplearning/ref/nnet.cnn.layer.imageinputlayer.html)
# Compute using the formula: $x\prime = 2 * \frac{x - \min{x}}{\max{x} - \min{x}} - 1$ taken from: https://stats.stackexchange.com/a/178629
def rescaleSymmetric(x):
    x = zerocenter(x)
    return 2 * (x - np.min(x, axis=(0, 1)))/np.ptp(x, axis=(0, 1)) - 1

def normalizeIms(x, normalization):
    if normalization == "zerocenter":
        func = zerocenter
    elif normalization == "rescale-symmetric":
        func = rescaleSymmetric
    else:
        return x
    return np.array(list(map(lambda im: func(im), x)))

In [11]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self,fraction, model):
        super(CustomCallback,self).__init__()
        self.fraction = fraction
        self.train_a = [];
        self.val_a =[];
        self.logPath = os.path.join(model, "log.txt")

        if not os.path.isdir(model):
            os.mkdir(model)

        with open(self.logPath,'w') as f:
            f.write('Starting of logging..\n')

        self.fig = plt.figure(figsize=(4,3))
        self.ax = plt.subplot(1,1,1)
        plt.ion()

    def on_train_begin(self,logs=None):
        self.fig.show()
        self.fig.canvas.draw()

    def on_train_end(self,logs=None):
        with open(self.logPath,'a') as f:
              f.write('End of logging..\n')
    
    def on_epoch_begin(self,epoch,logs=None):
        lr= tf.keras.backend.get_value(self.model.optimizer.lr)
        lr *= self.fraction
        tf.keras.backend.set_value(self.model.optimizer.lr,lr)
        with open(self.logPath,'a') as f:
            f.write('At epoch {:02d}, learning rate changed to {:.4f}\n'.format(epoch,lr))
    
    def on_epoch_end(self,epoch,logs=None):
        val_acc = logs.get('val_accuracy')
        train_acc = logs.get('accuracy')
        self.train_a.append(train_acc)
        self.val_a.append(val_acc)
        with open(self.logPath,'a') as f:
            f.write('At epoch {:02d}, training accuracy: {:.3f}, validation accuracy: {:.3f}\n'.format(epoch,train_acc,val_acc))
        self.ax.clear()
        self.ax.plot(range(1+epoch),self.train_a,label="Training")
        self.ax.plot(range(1+epoch),self.val_a,label="Validation")
        self.ax.set_xlabel('Epochs')
        self.ax.set_ylabel('Accuracy')
        self.ax.legend()
        self.fig.canvas.draw()
        self.fig.show()
        
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

## Load models

In [12]:
# xception base model
#from tensorflow.keras.applications.xception import Xception
#from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
#xceptionNetModel = Xception(weights='imagenet')
#xceptionNetModel.save("../Xception.hdf5")

xceptionNetModel = tf.keras.models.load_model('../Xception.hdf5')

# opticnet base model
opticNetModel = tf.keras.models.load_model('../Optic_net-4_classes-Kermany2018.hdf5')

# resnet base model
#from tensorflow.keras.applications.resnet50 import ResNet50
#from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
#resNetModel = ResNet50(weights='imagenet')
#resNetModel.save("../Resnet50.hdf5")

resNetModel = tf.keras.models.load_model('../Resnet50.hdf5')

In [13]:
conv_init = tf.keras.initializers.RandomNormal(mean=0., stddev=1.) # convolutional layer with zero mean and 1 sd
#batch_init = tf.keras.initializers.RandomNormal(mean=1., stddev=0.02) # non-sense here

octnet = keras.models.Sequential() # empty model

octnet.add(keras.layers.Conv2D(32, (7, 7), activation="relu", use_bias=True, input_shape=(227, 227, 3), kernel_initializer=conv_init))
octnet.add(keras.layers.BatchNormalization())
octnet.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

octnet.add(keras.layers.Conv2D(32, (7, 7), activation="relu", use_bias=True, kernel_initializer=conv_init))
octnet.add(keras.layers.BatchNormalization())
octnet.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

octnet.add(keras.layers.Conv2D(64, (5, 5), activation="relu", use_bias=True, kernel_initializer=conv_init))
octnet.add(keras.layers.BatchNormalization())
octnet.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

octnet.add(keras.layers.Conv2D(128, (5, 5), activation="relu", use_bias=True, kernel_initializer=conv_init))
octnet.add(keras.layers.BatchNormalization())
octnet.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

octnet.add(keras.layers.Conv2D(256, (3, 3), activation="relu", use_bias=True, kernel_initializer=conv_init))
octnet.add(keras.layers.BatchNormalization())
octnet.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

#octnet.add(keras.layers.Conv2D(512, (3, 3), activation="relu", use_bias=True))
#octnet.add(keras.layers.BatchNormalization())
#octnet.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

octnet.add(keras.layers.AveragePooling2D())
octnet.add(keras.layers.Flatten())
octnet.add(keras.layers.Dense(128))
octnet.add(keras.layers.Dropout(0.5))
octnet.add(keras.layers.Dense(32))
octnet.add(keras.layers.Dropout(0.5))
octnet.add(keras.layers.Dense(4))
octnet.add(keras.layers.Softmax())

octnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 221, 221, 32)      4736      
_________________________________________________________________
batch_normalization (BatchNo (None, 221, 221, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 104, 104, 32)      50208     
_________________________________________________________________
batch_normalization_1 (Batch (None, 104, 104, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 52, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        5

In [14]:
def emptyModelGenerator(model, newWeights=False, lastOnly=False):
    model = model.lower() # lower case model name
    if model == "resnet":
        x = resNetModel.output
        predictions = Dense(len(labels_list), activation='softmax')(x)
        newModel = Model(inputs=resNetModel.input, outputs=predictions)
        size = (224, 224)
        normalization = "zerocenter"
    elif model == "xception":
        x = xceptionNetModel.output
        predictions = Dense(len(labels_list), activation='softmax')(x)
        newModel = Model(inputs=xceptionNetModel.input, outputs=predictions)
        size = (299, 299)
        normalization = "rescale-symmetric"
    elif model == "opticnet":
        newModel = tf.keras.models.load_model('../Optic_net-4_classes-Kermany2018.hdf5')
        size = (224, 224)
        normalization = "rescale-symmetric"
    elif model == "octnet":
        newModel = tf.keras.models.clone_model(octnet)
        size = (227, 227)
        normalization = "zerocenter"
    if newWeights:
        newModel = tf.keras.models.clone_model(newModel)
    if lastOnly:
        newModel = lastDenseTrainable(newModel) # block all layer except the last dense ones
    else:
        newModel.trainable = True
    return newModel, size, normalization


def lastDenseTrainable(model):
    model.trainable = False
    for i in range(1, len(model.layers)):
        if model.layers[-i].__class__.__name__ == "Dense":
            model.layers[-i].trainable= True
        elif model.layers[-i].__class__.__name__ == "Activation":
            continue
        else:
            break
    return model
    

def testPredict(model, size, name=None, normalization=None):
    X_test = resizeIms(x_tst, size)
    if normalization:
        X_test = normalizeIms(X_test, normalization)
    X_test = np.array(X_test)
    Y_test = np.array(y_tst)
    
    prediction = model.predict(X_test)
    preds = np.argmax(prediction, axis=1)
    acc = sum(preds == Y_test) / len(Y_test)
    print(f'Test acc for {name if name else "model"}: {acc:.6f}')
    return acc
    

def computeConfussionMatrix(predictions, labels):
    num_labels = len(labels_list)
    cMatrix = np.zeros(shape=(num_labels, num_labels))
    for i in len(predictions):
        p = int(predictions[i])
        t = int(predictions[i])
        cMatrix[t, p] += 1
    print(cMatrix)
    ax = sns.heatmap(cMatrix, cmap="Blues", annot=True, fmt="d", xticklabels=labels_list, yticklabels=labels_list)
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title("Confusion matrix")
    plt.show();
    return cMatrix
    

In [15]:
models = ["xception", "resnet", "opticnet", "octnet"]

def printTrainableLayers(m):
    a = 0
    for l in m.layers:
        #l.trainable = True
        if l.trainable:
            #print(l.name, l.trainable)
            a += 1
    print(f"\nTotal: {len(m.layers)} trainable: {a}")

for mod in models:
    print("\n\n" + ("*"*50) + "\nModel: " + mod + '\n' + ("*"*50))
    m = emptyModelGenerator(mod, newWeights=False, lastOnly=True)[0]
    printTrainableLayers(m)



**************************************************
Model: xception
**************************************************

Total: 135 trainable: 2


**************************************************
Model: resnet
**************************************************

Total: 178 trainable: 2


**************************************************
Model: opticnet
**************************************************

Total: 276 trainable: 2


**************************************************
Model: octnet
**************************************************

Total: 23 trainable: 0


In [74]:
learning_rate = 0.001
momentum = 0.9
optim = lambda lr: Adam(learning_rate=lr)
#optim = lambda lr, momentum: SGD(learning_rate=lr, momentum=momentum)
epochs = 30
batch_size = 50
#batch_size = 128

In [75]:
import pandas as pd
results = pd.read_csv("Results_temp.csv")
results["normalization"] = None

In [78]:
#import pandas as pd
#res = pd.read_csv("Results_temp.csv")
#res.to_csv("Results.csv", index=False)

In [77]:
from sklearn.model_selection import train_test_split
import datetime

maxTrain = len(y_train)
for newWeights in [True, False]:
    tLayer = [False] if newWeights else [True, False] # do not train only last layer if new weights will be produced 
    for trainLastLayerOnly in tLayer:
        #for p in [0.01, 0.025, 0.05, 0.075, 0.09, 0.1, 0.25, 0.4, 0.5, 0.6, 0.75, 0.9, 1.0]:
        p = 0.25
        for learning_rate in [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]:
            #X_trn, X_tst, y_trn, y_tst
            if p < 1:
                X_t, _, y_t, _ = train_test_split(images, y_train, test_size=1-p, random_state=56789)
            else:
                X_t = images; y_t = y_train;
            print(f"Labels fraction: {sum(y_t == 1) / sum(y_train == 1)}")
            for net in ["resnet"]:
                print(f"Training {net} for {p}% of train size (aka {len(X_t)} images) with pretrained: {not newWeights} and onlyLastLayersTrained: {trainLastLayerOnly}...")
                model, size, normalization = emptyModelGenerator(net, newWeights=newWeights, lastOnly=trainLastLayerOnly)
                printTrainableLayers(model) # see if model is really well trained
                X_trn = resizeIms(X_t, size)
                #X_trn = normalizeIms(X_trn, normalization=normalization)
                X_val = resizeIms(x_val, size)
                #X_val = normalizeIms(X_val, normalization=normalization)
                log_dir = f"logs/{net}/fit/{p}trainSet_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
                tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
                optimizer = SGD(learning_rate=learning_rate, momentum=momentum) #Adam(learning_rate) # create new optimizers
                time_callback = TimeHistory()
                model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
                hist = model.fit(X_trn, y_t, epochs=epochs, validation_data = (X_val, y_validation), batch_size=batch_size,
                            shuffle=True, max_queue_size=20,
                            use_multiprocessing=True, workers=5, 
                            callbacks=[CustomCallback(fraction=0.9, model=net), tensorboard_callback, time_callback])
                trainTime = sum(time_callback.times)
                model.save(f"../{net}/{net}_{epochs}epochs_{p*maxTrain:.0f}_images_{newWeights}_newWeights_{trainLastLayerOnly}_lastLayerOnly_{normalization}_normalization")
                tstAcc = testPredict(model, size, name=net) #, normalization=normalization)
                results = results.append({
                    'model': net, 
                    'train set images': len(X_t), 
                    'pretrained': not newWeights, 
                    'pretrained dataset': None if newWeights or net == "octnet" else ("OCT2017,Srinivasan2014" if net == "opticnet" else "Imagenet"),
                    'Trained layers': "Last dense" if trainLastLayerOnly else "All", 
                    'epochs': epochs, 
                    'batch size': batch_size, 
                    'learning rate': f"{learning_rate}_{momentum}" if optimizer._name == "SGD" else learning_rate, 
                    'optimizer': optimizer._name,
                    'training time (seconds)': trainTime, 
                    'train accuracy': hist.history["accuracy"][-1], 
                    'train loss': hist.history["loss"][-1],
                    'validation accuracy': hist.history["val_accuracy"][-1], 
                    'validation loss': hist.history["val_loss"][-1], 
                    'test accuracy': tstAcc,
                    #'normalization': normalization
                }, ignore_index=True)
                results.to_csv("Results_temp.csv", index=False)
                del model
                del X_trn
                del X_val
                print("Done!\n" + '-'*50, end='\n\n')

Labels fraction: [0.25251758 0.24784303 0.25484667 0.24524141]
Training resnet for 0.25% of train size (aka 20871 images) with pretrained: False and onlyLastLayersTrained: False...

Total: 178 trainable: 178


<ipython-input-11-a21c27d31b2a>:20: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 1/30
418/418 [==============================] - 91s 203ms/step - loss: 1.2446 - accuracy: 0.4421 - val_loss: 1.5592 - val_accuracy: 0.2500


<ipython-input-11-a21c27d31b2a>:48: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 2/30
418/418 [==============================] - 83s 199ms/step - loss: 1.0402 - accuracy: 0.6094 - val_loss: 1.3081 - val_accuracy: 0.4375
Epoch 3/30
418/418 [==============================] - 84s 201ms/step - loss: 0.8593 - accuracy: 0.6885 - val_loss: 1.6566 - val_accuracy: 0.2812
Epoch 4/30
418/418 [==============================] - 84s 201ms/step - loss: 0.7628 - accuracy: 0.7175 - val_loss: 2.3735 - val_accuracy: 0.2500
Epoch 5/30
418/418 [==============================] - 84s 200ms/step - loss: 0.7476 - accuracy: 0.7314 - val_loss: 1.0548 - val_accuracy: 0.6562
Epoch 6/30
418/418 [==============================] - 83s 200ms/step - loss: 0.6714 - accuracy: 0.7693 - val_loss: 1.1976 - val_accuracy: 0.5625
Epoch 7/30
418/418 [==============================] - 84s 200ms/step - loss: 0.6419 - accuracy: 0.7859 - val_loss: 0.8864 - val_accuracy: 0.6250
Epoch 8/30
418/418 [==============================] - 84s 200ms/step - loss: 0.5699 - accuracy: 0.8087 - val_loss: 1.6507 - val_ac

...

Epoch 2/30
418/418 [==============================] - 83s 198ms/step - loss: 1.3252 - accuracy: 0.7585 - val_loss: 1.3348 - val_accuracy: 0.5000
Epoch 3/30
418/418 [==============================] - 83s 199ms/step - loss: 1.2777 - accuracy: 0.7639 - val_loss: 1.3105 - val_accuracy: 0.5000
Epoch 4/30
418/418 [==============================] - 83s 199ms/step - loss: 1.2386 - accuracy: 0.7602 - val_loss: 1.2899 - val_accuracy: 0.5000
Epoch 5/30
418/418 [==============================] - 83s 199ms/step - loss: 1.2054 - accuracy: 0.7602 - val_loss: 1.2719 - val_accuracy: 0.5000
Epoch 6/30
418/418 [==============================] - 83s 199ms/step - loss: 1.1774 - accuracy: 0.7600 - val_loss: 1.2561 - val_accuracy: 0.5000
Epoch 7/30
418/418 [==============================] - 83s 198ms/step - loss: 1.1517 - accuracy: 0.7594 - val_loss: 1.2427 - val_accuracy: 0.5000
Epoch 8/30
418/418 [==============================] - 83s 198ms/step - loss: 1.1288 - accuracy: 0.7634 - val_loss: 1.2307 - val_ac

<ipython-input-11-a21c27d31b2a>:20: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 1/30
418/418 [==============================] - 93s 206ms/step - loss: 1.3736 - accuracy: 0.2361 - val_loss: 1.3586 - val_accuracy: 0.4688


<ipython-input-11-a21c27d31b2a>:48: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  self.fig.show()


Epoch 2/30
418/418 [==============================] - 83s 200ms/step - loss: 1.3621 - accuracy: 0.2384 - val_loss: 1.3525 - val_accuracy: 0.7188
Epoch 3/30
418/418 [==============================] - 83s 199ms/step - loss: 1.3518 - accuracy: 0.6415 - val_loss: 1.3471 - val_accuracy: 0.5000
Epoch 4/30
418/418 [==============================] - 83s 199ms/step - loss: 1.3429 - accuracy: 0.5742 - val_loss: 1.3423 - val_accuracy: 0.5000
Epoch 5/30
418/418 [==============================] - 83s 199ms/step - loss: 1.3345 - accuracy: 0.5836 - val_loss: 1.3381 - val_accuracy: 0.5000
Epoch 6/30
418/418 [==============================] - 84s 200ms/step - loss: 1.3276 - accuracy: 0.6940 - val_loss: 1.3342 - val_accuracy: 0.7500
Epoch 7/30
418/418 [==============================] - 84s 200ms/step - loss: 1.3211 - accuracy: 0.8974 - val_loss: 1.3308 - val_accuracy: 0.7500
Epoch 8/30
418/418 [==============================] - 83s 199ms/step - loss: 1.3153 - accuracy: 0.8992 - val_loss: 1.3277 - val_ac